# Imports

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()
pd.options.display.max_columns = None

# Lecture des données

In [38]:
fifa15 =pd.read_csv('../data/FIFA/players_15.csv')
col_int = fifa15.columns[(fifa15.dtypes==np.int64)& (fifa15.columns!='sofifa_id')]
col_str = fifa15.columns[(fifa15.dtypes!=np.int64) & (fifa15.dtypes!=np.float64)]
dict_type = {**{x: np.int8 for x in col_int},\
    **{x: str for x in col_str}}


/Users/pierreadda/opt/anaconda3/envs/foot_env/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (104) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [39]:
fifa15 =pd.read_csv('../data/FIFA/players_15.csv',dtype=dict_type)
fifa16 =pd.read_csv('../data/FIFA/players_16.csv',dtype=dict_type)
fifa17 =pd.read_csv('../data/FIFA/players_17.csv',dtype=dict_type)
fifa18 =pd.read_csv('../data/FIFA/players_18.csv',dtype=dict_type)
fifa19 =pd.read_csv('../data/FIFA/players_19.csv',dtype=dict_type)
fifa20 =pd.read_csv('../data/FIFA/players_20.csv',dtype=dict_type)
fifa21 =pd.read_csv('../data/FIFA/players_21.csv',dtype=dict_type)
fifa22 =pd.read_csv('../data/FIFA/players_22.csv',dtype=dict_type)

In [41]:
clubs = pd.read_csv('../data/games/clubs.csv', index_col = 0)
leagues = pd.read_csv('../data/games/leagues.csv', index_col = 0)
games = pd.read_csv('../data/games/games.csv', index_col = 0,
                   dtype = {'game_id' : np.int32, 'season' : np.int16, 'home_club_id' : np.int32, 'home_club_id' : np.int32,
                           'home_club_goals' : np.int8, 'away_club_goals' : np.int8})

players = pd.read_csv('../data/games/players.csv', index_col = 0,
                     dtype = {'player_id' : np.int32, 'player_club_id' : np.int16})

appearance = pd.read_csv('../data/games/appearance.csv', index_col = 0)
corr = pd.read_csv('../data/table_correspondance/table_correspondance_finale.csv', index_col = 0)
player_IDs = corr.player_id.unique()


In [42]:
appearance_select =\
pd.read_csv('../data/training_data/appearance_select.csv', index_col = 0, \
            dtype = {'game_id' : np.int32, 'player_id' : np.int32,'player_club_id' : np.int16, 'minutes_played' : np.int8,\
                    'yellow_cards' : np.int8,'red_cards' : np.int8,'goals' : np.int8,'assists' : np.int8,
                    'max_time' : np.int16,'time_ratio' : np.float16, 'season' : np.int8})

appearance_known_players =\
pd.read_csv('../data/training_data/appearance_known_players.csv', index_col = 0,\
            dtype = {'game_id' : np.int32, 'player_id' : np.int32,'player_club_id' : np.int16, 'minutes_played' : np.int8,\
                    'yellow_cards' : np.int8,'red_cards' : np.int8,'goals' : np.int8,'assists' : np.int8,
                    'max_time' : np.int16,'time_ratio' : np.float16, 'season' : np.int8})

train1 = pd.read_csv('../data/training_data/train1.csv', index_col = 0)

#### colonnes des statistiques fifa à sélectionner

In [48]:
#col_int
#fifa15.loc[:,['skill_moves','international_reputation']]

In [49]:
stats_col = ['overall' ,'potential','skill_moves','international_reputation','weak_foot','pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic',
       'attacking_crossing','attacking_heading_accuracy','skill_curve','skill_fk_accuracy', 'movement_agility',
       'movement_reactions', 'movement_balance', 'power_shot_power','power_jumping', 'power_stamina','mentality_aggression',
       'mentality_positioning', 'mentality_penalties','mentality_composure']


GK_stats_col = ['overall' ,'potential','skill_moves','international_reputation','weak_foot','attacking_crossing','attacking_heading_accuracy','skill_curve','skill_fk_accuracy', 
                'movement_agility','movement_reactions', 'movement_balance', 'power_shot_power','power_jumping', 'power_stamina',
                'mentality_aggression','mentality_positioning', 'mentality_penalties','goalkeeping_diving', 
                'goalkeeping_speed']

# Création d'un profil moyen de joueurs d'attaque/défense/milieu/GK

In [51]:
# Arrangeons un petit peu le df players
players.loc[players.player_id == 290801,'position'] = 'Defender'
players.loc[players.player_id == 290801,'sub_position'] = 'Centre-Back'
t  = players.loc[players.position == '0','sub_position']
players.loc[t.index,'position'] = t
players.drop(columns =
             ['name','pretty_name','country_of_birth','country_of_citizenship','foot','height_in_cm','market_value_in_gbp' ,'highest_market_value_in_gbp' ,'url'],
            inplace = True)
players.position = players.position.apply(lambda x: x.lower() )


In [52]:
# liste des joueurs de la table de correspondance pouvant être trouvés dans fifa22:
players22_ID = corr.loc[corr.which_fifa==22,'player_id'].values

# faisons un df ne contenant que ces joueurs
players22 = players.loc[players.player_id.isin(players22_ID),:].reset_index(drop=True).copy()

In [53]:
players22.position.unique()

array(['defender', 'midfield', 'goalkeeper', 'attack'], dtype=object)

In [54]:
GK22 = players22.loc[players22.position == 'goalkeeper',:].reset_index(drop=True).copy()
MID22 = players22.loc[players22.position == 'midfield',:].reset_index(drop=True).copy()
DEF22 = players22.loc[players22.position == 'defender',:].reset_index(drop=True).copy()
ATK22 = players22.loc[players22.position == 'attack',:].reset_index(drop=True).copy()

## GK

In [55]:
def seek_statGK22(x):
    sofID = corr.loc[(corr.player_id == x) & (corr.which_fifa == 22),'sofifa_id'].values[0]
    return fifa22.loc[fifa22.sofifa_id==sofID,GK_stats_col]

tt=GK22.player_id.progress_apply(seek_statGK22)
all_GK = pd.concat(tt.values.tolist())
####====>
avg_GK = all_GK.mean(axis=0)

100%|██████████| 880/880 [00:01<00:00, 662.72it/s]


## ATK, DEF, & MID

In [56]:
def seek_stat22(x):
    sofID = corr.loc[(corr.player_id == x) & (corr.which_fifa == 22),'sofifa_id'].values[0]
    return fifa22.loc[fifa22.sofifa_id==sofID,stats_col]

tt=ATK22.player_id.progress_apply(seek_stat22)
all_ATK = pd.concat(tt.values.tolist())
avg_ATK = all_ATK.mean(axis=0)


tt=DEF22.player_id.progress_apply(seek_stat22)
all_DEF = pd.concat(tt.values.tolist())
avg_DEF = all_DEF.mean(axis=0)

tt=MID22.player_id.progress_apply(seek_stat22)
all_MID = pd.concat(tt.values.tolist())
avg_MID = all_MID.mean(axis=0)

100%|██████████| 2387/2387 [00:03<00:00, 653.47it/s]


# stats des GK

In [57]:
#On commence par retirer les matchs ou il n'y a pas de GK parce que c'est bizarre

#Identification des quelques matchs sans GK
bad_game_ids = train1.loc[train1.GK < 0.8,'game_id'].unique()
print(len(bad_game_ids))

appearance_select = \
appearance_select.drop(appearance_select.loc[appearance_select.game_id.isin(bad_game_ids),:].index).reset_index(drop = True)

appearance_known_players = \
appearance_known_players.drop(appearance_known_players.loc[appearance_known_players.game_id.isin(bad_game_ids),:].index).reset_index(drop = True)

train1 = train1.drop(train1.loc[train1.game_id.isin(bad_game_ids),:].index).reset_index(drop = True)


203


In [58]:
# On créé un dataframe à partir de appearance_known_players avec uniquement les GKs

known_GK = appearance_known_players.loc[appearance_known_players.position == 'goalkeeper',
                                        ['game_id','player_club_id', 'time_ratio','which_fifa','player_id']].reset_index(drop = True).copy()
#On garde en tête qu'on a pu perdre des lignes voire des équipes dans des matches

In [59]:
# Test d'une ligne d'un apply:
#On extrait les stats (GK_stats_col) d'un player_id dans un dataframe fifa désigné par which_fifa 

x = known_GK.iloc[3,:]
locs = locals()
sofifa = corr.loc[(corr.player_id == x['player_id']) & (corr.which_fifa == x['which_fifa']), 'sofifa_id'].values[0]
FIFA = locs['fifa'+str(int(x['which_fifa']))]
FIFA.loc[FIFA.sofifa_id==sofifa,GK_stats_col]


,overall,potential,skill_moves,international_reputation,weak_foot,attacking_crossing,attacking_heading_accuracy,skill_curve,skill_fk_accuracy,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,mentality_aggression,mentality_positioning,mentality_penalties,goalkeeping_diving,goalkeeping_speed
263,80,84,1,2,4,25,25,25,25,42,79,37,28,76,45,30,25,35,81,49.0


In [60]:
#On extrait les stats (GK_stats_col) d'un player_id dans un dataframe fifa désigné par which_fifa 

locs = locals()
def GK_stats(x):
    sofifa = corr.loc[(corr.player_id == x['player_id']) & (corr.which_fifa == x['which_fifa']), 'sofifa_id'].values[0]
    FIFA = locs['fifa'+str(int(x['which_fifa']))]
    
    return FIFA.loc[FIFA.sofifa_id==sofifa,GK_stats_col]
    
    
    
GK_stats = known_GK.progress_apply(lambda x: GK_stats(x)
                          ,axis = 1
                         )
GK_stats = pd.concat(GK_stats.values.tolist()).reset_index(drop = True)

100%|██████████| 58094/58094 [01:39<00:00, 586.63it/s]


In [61]:
#on renomme les colonnes de GK_stats pour ajouter GK au début!
GK_stats.rename(columns = {x:'GK_'+x for x in GK_stats.columns}, inplace = True)
GK_stats.head(1)

,GK_overall,GK_potential,GK_skill_moves,GK_international_reputation,GK_weak_foot,GK_attacking_crossing,GK_attacking_heading_accuracy,GK_skill_curve,GK_skill_fk_accuracy,GK_movement_agility,GK_movement_reactions,GK_movement_balance,GK_power_shot_power,GK_power_jumping,GK_power_stamina,GK_mentality_aggression,GK_mentality_positioning,GK_mentality_penalties,GK_goalkeeping_diving,GK_goalkeeping_speed
0,80,84,1,2,4,25,25,25,25,42,79,37,28,76,45,30,25,35,81,49.0


##### Moyenne pondérée des GK _que l'on connait_

In [62]:
# On multiplie les stats par le time_ratio (moyenne pondérée):
GK_stats = GK_stats.multiply(known_GK.time_ratio,axis = 0)

In [64]:
# On ajoute GK_stats à known_GK
if known_GK.shape[1]==5:
    known_GK = pd.concat([known_GK , GK_stats],axis = 1)

In [65]:
# On fait la somme des stats pondérées des gardiens de but par équipes et par match
known_GK = known_GK.groupby(['game_id','player_club_id'])[np.append(['time_ratio'],GK_stats.columns)].sum().reset_index()

In [66]:
# Puis on divise par la somme des time_ratios (toujours la moyenne pondérée):

known_GK.loc[:,GK_stats.columns] = known_GK.loc[:,GK_stats.columns].divide(known_GK.time_ratio, axis = 0)


In [67]:
# à ce stade on a des lignes pour les GK utilisables pour l'entrainement du modèle.
# Mais on sait que quand on n'a selectionné que les GK que l'on connait, on a perdu quelques lignes en route. 
# En l'occurence, comme il n'y a en général qu'un GK par match et par équipe, on a dû perdre des équipes. On vérifie:

team_count = known_GK.groupby('game_id')['player_club_id'].unique().reset_index()
print(team_count.loc[team_count.player_club_id.apply(len)<2,'game_id'].shape[0])
#il en manque 1576

1576


In [70]:
# On créée le fichier d'entrainement avec pour l'instant seulement les GK.
# On fera la même chose pour les ATK, DEF et MF, puis on mergera tout ça
train_GK = train1.loc[:,['game_id','player_club_id','GK']].copy()

In [71]:
train_GK = pd.merge(train_GK , known_GK.drop(columns = ['time_ratio']), \
         on = ['player_club_id','game_id'], how = 'left')


In [72]:
train_GK.loc[train_GK.GK_overall.isnull(),:].head(2)

,game_id,player_club_id,GK,GK_overall,GK_potential,GK_skill_moves,GK_international_reputation,GK_weak_foot,GK_attacking_crossing,GK_attacking_heading_accuracy,GK_skill_curve,GK_skill_fk_accuracy,GK_movement_agility,GK_movement_reactions,GK_movement_balance,GK_power_shot_power,GK_power_jumping,GK_power_stamina,GK_mentality_aggression,GK_mentality_positioning,GK_mentality_penalties,GK_goalkeeping_diving,GK_goalkeeping_speed
2,2458528,338,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
628,2460301,281,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [73]:
# le pd.merge a créé des lignes de NaN là où il nous manque des GK. on remplace les NaN par les valeurs de GK moyen (stratégie d'imputing)
train_GK.loc[train_GK.GK_overall.isnull(),GK_stats.columns] = avg_GK.values
train_GK.loc[train_GK.GK_overall == avg_GK.values[0],:].head()

,game_id,player_club_id,GK,GK_overall,GK_potential,GK_skill_moves,GK_international_reputation,GK_weak_foot,GK_attacking_crossing,GK_attacking_heading_accuracy,GK_skill_curve,GK_skill_fk_accuracy,GK_movement_agility,GK_movement_reactions,GK_movement_balance,GK_power_shot_power,GK_power_jumping,GK_power_stamina,GK_mentality_aggression,GK_mentality_positioning,GK_mentality_penalties,GK_goalkeeping_diving,GK_goalkeeping_speed
2,2458528,338,1.0,68.125,72.8125,1.0,1.18457,2.568359,13.9375,14.117188,14.734375,13.835938,41.15625,62.2500,40.93750,48.71875,57.6875,30.296875,26.21875,10.523438,19.640625,68.6875,38.606818
68,2459617,200,1.0,68.125,68.7500,1.0,1.00000,2.000000,25.0000,25.000000,25.000000,25.000000,45.12500,69.9375,28.09375,36.12500,51.5625,36.312500,31.93750,25.000000,23.000000,62.0000,36.397949
628,2460301,281,1.0,68.125,72.8125,1.0,1.18457,2.568359,13.9375,14.117188,14.734375,13.835938,41.15625,62.2500,40.93750,48.71875,57.6875,30.296875,26.21875,10.523438,19.640625,68.6875,38.606818
900,2460757,24,1.0,68.125,72.8125,1.0,1.18457,2.568359,13.9375,14.117188,14.734375,13.835938,41.15625,62.2500,40.93750,48.71875,57.6875,30.296875,26.21875,10.523438,19.640625,68.6875,38.606818
918,2460766,24,1.0,68.125,72.8125,1.0,1.18457,2.568359,13.9375,14.117188,14.734375,13.835938,41.15625,62.2500,40.93750,48.71875,57.6875,30.296875,26.21875,10.523438,19.640625,68.6875,38.606818


In [75]:
train_GK.to_csv('data/training_data/train_GK.csv')

# stats des Attaquants

In [76]:
known_ATK = appearance_known_players.loc[appearance_known_players.position == 'attack',
                                        ['game_id','player_club_id', 'time_ratio','which_fifa','player_id']].reset_index(drop = True).copy()


In [77]:
locs = locals()
def player_stats(x):
    sofifa = corr.loc[(corr.player_id == x['player_id']) & (corr.which_fifa == x['which_fifa']), 'sofifa_id'].values[0]
    FIFA = locs['fifa'+str(int(x['which_fifa']))]
    
    return FIFA.loc[FIFA.sofifa_id==sofifa,stats_col]
    
    
    
ATK_stats = known_ATK.progress_apply(lambda x: player_stats(x)
                          ,axis = 1
                         )
ATK_stats = pd.concat(ATK_stats.values.tolist()).reset_index(drop = True)

100%|██████████| 220216/220216 [06:18<00:00, 581.19it/s]


In [78]:
ATK_stats.rename(columns = {x:'ATK_'+x for x in ATK_stats.columns}, inplace = True)
ATK_stats.head(1)

,ATK_overall,ATK_potential,ATK_skill_moves,ATK_international_reputation,ATK_weak_foot,ATK_pace,ATK_shooting,ATK_passing,ATK_dribbling,ATK_defending,ATK_physic,ATK_attacking_crossing,ATK_attacking_heading_accuracy,ATK_skill_curve,ATK_skill_fk_accuracy,ATK_movement_agility,ATK_movement_reactions,ATK_movement_balance,ATK_power_shot_power,ATK_power_jumping,ATK_power_stamina,ATK_mentality_aggression,ATK_mentality_positioning,ATK_mentality_penalties,ATK_mentality_composure
0,56,63,3,1,3,52.0,55.0,40.0,55.0,30.0,64.0,27,58,33,34,55,54,45,60,68,65,51,48,37,NaN


In [79]:
ATK_stats = ATK_stats.multiply(known_ATK.time_ratio,axis = 0)

In [80]:
# On ajoute ATK_stats à known_ATK
if known_ATK.shape[1]==5:
    known_ATK = pd.concat([known_ATK , ATK_stats],axis = 1)

In [81]:
# On fait la somme des stats pondérées des attaquants par équipes et par match
known_ATK = known_ATK.groupby(['game_id','player_club_id'])[np.append(['time_ratio'],ATK_stats.columns)].sum().reset_index()

In [82]:
# Puis on divise par la somme des time_ratios (toujours la moyenne pondérée):AT
known_ATK.loc[:,ATK_stats.columns] = known_ATK.loc[:,ATK_stats.columns].divide(known_ATK.time_ratio, axis = 0)

In [83]:
# On créée le fichier d'entrainement avec les ATK:
train_ATK = train1.loc[:,['game_id','player_club_id','attack']].copy()

In [84]:
train_ATK = pd.merge(train_ATK , known_ATK.drop(columns = ['time_ratio']), \
         on = ['player_club_id','game_id'], how = 'left')


In [85]:
# Reste deux cas de figures: soit un match n'a pas d'attaquant (c'est possible!), soit on ne connait pas les attaquants!
#S'il n'y a pas d'attaquants, alors attack=0 et on met les stats à 0
train_ATK.loc[train_ATK.attack == 0,ATK_stats.columns]=0
train_ATK.loc[train_ATK.attack == 0,ATK_stats.columns].head(1)

,ATK_overall,ATK_potential,ATK_skill_moves,ATK_international_reputation,ATK_weak_foot,ATK_pace,ATK_shooting,ATK_passing,ATK_dribbling,ATK_defending,ATK_physic,ATK_attacking_crossing,ATK_attacking_heading_accuracy,ATK_skill_curve,ATK_skill_fk_accuracy,ATK_movement_agility,ATK_movement_reactions,ATK_movement_balance,ATK_power_shot_power,ATK_power_jumping,ATK_power_stamina,ATK_mentality_aggression,ATK_mentality_positioning,ATK_mentality_penalties,ATK_mentality_composure
5144,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [86]:
#Pour les matchs où on ne connait aucun attaquant, on ajoute l'attaquant moyen:
index = train_ATK.loc[train_ATK.ATK_overall.isnull(),:].index
train_ATK.loc[index,ATK_stats.columns] = avg_ATK.values
train_ATK.loc[index,:].head()

,game_id,player_club_id,attack,ATK_overall,ATK_potential,ATK_skill_moves,ATK_international_reputation,ATK_weak_foot,ATK_pace,ATK_shooting,ATK_passing,ATK_dribbling,ATK_defending,ATK_physic,ATK_attacking_crossing,ATK_attacking_heading_accuracy,ATK_skill_curve,ATK_skill_fk_accuracy,ATK_movement_agility,ATK_movement_reactions,ATK_movement_balance,ATK_power_shot_power,ATK_power_jumping,ATK_power_stamina,ATK_mentality_aggression,ATK_mentality_positioning,ATK_mentality_penalties,ATK_mentality_composure
654,2460634,39,1.000000,69.3125,73.9375,3.015625,1.219727,3.251953,74.459947,66.93662,60.317342,69.628081,34.18794,63.613996,57.84375,59.34375,60.09375,51.40625,72.375,65.5,69.1875,70.0625,65.375,65.8125,54.3125,68.125,63.1875,64.952025
1485,2469526,1465,2.000122,69.3125,73.9375,3.015625,1.219727,3.251953,74.459947,66.93662,60.317342,69.628081,34.18794,63.613996,57.84375,59.34375,60.09375,51.40625,72.375,65.5,69.1875,70.0625,65.375,65.8125,54.3125,68.125,63.1875,64.952025
3809,2480143,2424,1.500122,69.3125,73.9375,3.015625,1.219727,3.251953,74.459947,66.93662,60.317342,69.628081,34.18794,63.613996,57.84375,59.34375,60.09375,51.40625,72.375,65.5,69.1875,70.0625,65.375,65.8125,54.3125,68.125,63.1875,64.952025
3832,2480165,2424,1.355469,69.3125,73.9375,3.015625,1.219727,3.251953,74.459947,66.93662,60.317342,69.628081,34.18794,63.613996,57.84375,59.34375,60.09375,51.40625,72.375,65.5,69.1875,70.0625,65.375,65.8125,54.3125,68.125,63.1875,64.952025
6463,2505069,2451,1.000000,69.3125,73.9375,3.015625,1.219727,3.251953,74.459947,66.93662,60.317342,69.628081,34.18794,63.613996,57.84375,59.34375,60.09375,51.40625,72.375,65.5,69.1875,70.0625,65.375,65.8125,54.3125,68.125,63.1875,64.952025


In [88]:
train_ATK.to_csv('data/training_data/train_ATK.csv')

# Stats des défenseurs

In [89]:
known_DEF = appearance_known_players.loc[appearance_known_players.position == 'defender',
                                        ['game_id','player_club_id', 'time_ratio','which_fifa','player_id']].reset_index(drop = True).copy()


In [90]:
DEF_stats = known_DEF.progress_apply(lambda x: player_stats(x)
                          ,axis = 1
                         )
DEF_stats = pd.concat(DEF_stats.values.tolist()).reset_index(drop = True)

100%|██████████| 261520/261520 [07:29<00:00, 581.63it/s]


In [91]:
DEF_stats.rename(columns = {x:'DEF_'+x for x in DEF_stats.columns}, inplace = True)
DEF_stats.head(1)

,DEF_overall,DEF_potential,DEF_skill_moves,DEF_international_reputation,DEF_weak_foot,DEF_pace,DEF_shooting,DEF_passing,DEF_dribbling,DEF_defending,DEF_physic,DEF_attacking_crossing,DEF_attacking_heading_accuracy,DEF_skill_curve,DEF_skill_fk_accuracy,DEF_movement_agility,DEF_movement_reactions,DEF_movement_balance,DEF_power_shot_power,DEF_power_jumping,DEF_power_stamina,DEF_mentality_aggression,DEF_mentality_positioning,DEF_mentality_penalties,DEF_mentality_composure
0,68,74,2,1,2,72.0,62.0,66.0,67.0,66.0,68.0,68,61,65,48,68,69,68,65,67,75,74,65,50,NaN


In [92]:
DEF_stats = DEF_stats.multiply(known_DEF.time_ratio,axis = 0)

In [93]:
if known_DEF.shape[1]==5:
    known_DEF = pd.concat([known_DEF , DEF_stats],axis = 1)

In [94]:
known_DEF = known_DEF.groupby(['game_id','player_club_id'])[np.append(['time_ratio'],DEF_stats.columns)].sum().reset_index()

In [95]:
known_DEF.loc[:,DEF_stats.columns] = known_DEF.loc[:,DEF_stats.columns].divide(known_DEF.time_ratio, axis = 0)

In [96]:
train_DEF = train1.loc[:,['game_id','player_club_id','defense']].copy()

In [97]:
train_DEF = pd.merge(train_DEF , known_DEF.drop(columns = ['time_ratio']), \
         on = ['player_club_id','game_id'], how = 'left')


In [98]:
train_DEF.loc[train_DEF.defense == 0,DEF_stats.columns]=0
train_DEF.loc[train_DEF.defense == 0,:].head(1)

,game_id,player_club_id,defense,DEF_overall,DEF_potential,DEF_skill_moves,DEF_international_reputation,DEF_weak_foot,DEF_pace,DEF_shooting,DEF_passing,DEF_dribbling,DEF_defending,DEF_physic,DEF_attacking_crossing,DEF_attacking_heading_accuracy,DEF_skill_curve,DEF_skill_fk_accuracy,DEF_movement_agility,DEF_movement_reactions,DEF_movement_balance,DEF_power_shot_power,DEF_power_jumping,DEF_power_stamina,DEF_mentality_aggression,DEF_mentality_positioning,DEF_mentality_penalties,DEF_mentality_composure
51697,3432158,4294,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [99]:
index = train_DEF.loc[train_DEF.DEF_overall.isnull(),:].index
train_DEF.loc[index,DEF_stats.columns] = avg_DEF.values
train_DEF.loc[index,:].head()

,game_id,player_club_id,defense,DEF_overall,DEF_potential,DEF_skill_moves,DEF_international_reputation,DEF_weak_foot,DEF_pace,DEF_shooting,DEF_passing,DEF_dribbling,DEF_defending,DEF_physic,DEF_attacking_crossing,DEF_attacking_heading_accuracy,DEF_skill_curve,DEF_skill_fk_accuracy,DEF_movement_agility,DEF_movement_reactions,DEF_movement_balance,DEF_power_shot_power,DEF_power_jumping,DEF_power_stamina,DEF_mentality_aggression,DEF_mentality_positioning,DEF_mentality_penalties,DEF_mentality_composure
6964,2517535,339,3.0,69.1875,73.75,2.292969,1.176758,2.853516,65.916942,43.044469,56.354282,60.15068,67.382212,70.403528,54.4375,63.84375,47.5625,40.28125,61.21875,64.875,61.65625,55.40625,70.75,68.625,68.4375,47.09375,44.6875,62.214994
15470,2646302,2451,1.0,69.1875,73.75,2.292969,1.176758,2.853516,65.916942,43.044469,56.354282,60.15068,67.382212,70.403528,54.4375,63.84375,47.5625,40.28125,61.21875,64.875,61.65625,55.40625,70.75,68.625,68.4375,47.09375,44.6875,62.214994
20976,2722891,524,1.0,69.1875,73.75,2.292969,1.176758,2.853516,65.916942,43.044469,56.354282,60.15068,67.382212,70.403528,54.4375,63.84375,47.5625,40.28125,61.21875,64.875,61.65625,55.40625,70.75,68.625,68.4375,47.09375,44.6875,62.214994
44680,3221089,36,1.0,69.1875,73.75,2.292969,1.176758,2.853516,65.916942,43.044469,56.354282,60.15068,67.382212,70.403528,54.4375,63.84375,47.5625,40.28125,61.21875,64.875,61.65625,55.40625,70.75,68.625,68.4375,47.09375,44.6875,62.214994
44698,3221098,36,1.0,69.1875,73.75,2.292969,1.176758,2.853516,65.916942,43.044469,56.354282,60.15068,67.382212,70.403528,54.4375,63.84375,47.5625,40.28125,61.21875,64.875,61.65625,55.40625,70.75,68.625,68.4375,47.09375,44.6875,62.214994


In [100]:
train_DEF.to_csv('data/training_data/train_DEF.csv')

# Stats des milieux de terrain

In [101]:
known_MF = appearance_known_players.loc[appearance_known_players.position == 'midfield',
                                        ['game_id','player_club_id', 'time_ratio','which_fifa','player_id']].reset_index(drop = True).copy()


In [102]:
MF_stats = known_MF.progress_apply(lambda x: player_stats(x)
                          ,axis = 1
                         )
MF_stats = pd.concat(MF_stats.values.tolist()).reset_index(drop = True)

100%|██████████| 241961/241961 [06:53<00:00, 585.64it/s]


In [103]:
MF_stats.rename(columns = {x:'MF_'+x for x in MF_stats.columns}, inplace = True)
MF_stats.head(1)

,MF_overall,MF_potential,MF_skill_moves,MF_international_reputation,MF_weak_foot,MF_pace,MF_shooting,MF_passing,MF_dribbling,MF_defending,MF_physic,MF_attacking_crossing,MF_attacking_heading_accuracy,MF_skill_curve,MF_skill_fk_accuracy,MF_movement_agility,MF_movement_reactions,MF_movement_balance,MF_power_shot_power,MF_power_jumping,MF_power_stamina,MF_mentality_aggression,MF_mentality_positioning,MF_mentality_penalties,MF_mentality_composure
0,82,82,3,3,3,66.0,81.0,84.0,79.0,52.0,70.0,86,60,87,89,72,78,61,82,70,86,57,80,80,81.0


In [104]:
MF_stats = MF_stats.multiply(known_MF.time_ratio,axis = 0)

In [105]:
if known_MF.shape[1]==5:
    known_MF = pd.concat([known_MF , MF_stats],axis = 1)

In [106]:
known_MF = known_MF.groupby(['game_id','player_club_id'])[np.append(['time_ratio'],MF_stats.columns)].sum().reset_index()

In [107]:
known_MF.loc[:,MF_stats.columns] = known_MF.loc[:,MF_stats.columns].divide(known_MF.time_ratio, axis = 0)

In [108]:
train_MF = train1.loc[:,['game_id','player_club_id','midfield']].copy()

In [109]:
train_MF = pd.merge(train_MF , known_MF.drop(columns = ['time_ratio']), \
         on = ['player_club_id','game_id'], how = 'left')


In [110]:
train_MF.loc[train_MF.midfield == 0,MF_stats.columns]=0
train_MF.loc[train_MF.midfield == 0,:].head(1)

,game_id,player_club_id,midfield,MF_overall,MF_potential,MF_skill_moves,MF_international_reputation,MF_weak_foot,MF_pace,MF_shooting,MF_passing,MF_dribbling,MF_defending,MF_physic,MF_attacking_crossing,MF_attacking_heading_accuracy,MF_skill_curve,MF_skill_fk_accuracy,MF_movement_agility,MF_movement_reactions,MF_movement_balance,MF_power_shot_power,MF_power_jumping,MF_power_stamina,MF_mentality_aggression,MF_mentality_positioning,MF_mentality_penalties,MF_mentality_composure
9796,2581814,18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [111]:
index = train_MF.loc[train_MF.MF_overall.isnull(),:].index
train_MF.loc[index,MF_stats.columns] = avg_MID.values
train_MF.loc[index,:].head() 

,game_id,player_club_id,midfield,MF_overall,MF_potential,MF_skill_moves,MF_international_reputation,MF_weak_foot,MF_pace,MF_shooting,MF_passing,MF_dribbling,MF_defending,MF_physic,MF_attacking_crossing,MF_attacking_heading_accuracy,MF_skill_curve,MF_skill_fk_accuracy,MF_movement_agility,MF_movement_reactions,MF_movement_balance,MF_power_shot_power,MF_power_jumping,MF_power_stamina,MF_mentality_aggression,MF_mentality_positioning,MF_mentality_penalties,MF_mentality_composure
986,2460800,24,1.000000,69.25,74.25,2.826172,1.212891,3.148438,65.762044,59.722245,66.481357,69.112275,58.764139,66.000838,60.53125,54.96875,61.0,56.5,70.0625,66.4375,70.625,66.875,63.59375,71.0,64.375,61.9375,56.25,66.7323
2629,2475409,3368,1.000000,69.25,74.25,2.826172,1.212891,3.148438,65.762044,59.722245,66.481357,69.112275,58.764139,66.000838,60.53125,54.96875,61.0,56.5,70.0625,66.4375,70.625,66.875,63.59375,71.0,64.375,61.9375,56.25,66.7323
2723,2475456,3368,0.999756,69.25,74.25,2.826172,1.212891,3.148438,65.762044,59.722245,66.481357,69.112275,58.764139,66.000838,60.53125,54.96875,61.0,56.5,70.0625,66.4375,70.625,66.875,63.59375,71.0,64.375,61.9375,56.25,66.7323
6234,2498721,294,1.133301,69.25,74.25,2.826172,1.212891,3.148438,65.762044,59.722245,66.481357,69.112275,58.764139,66.000838,60.53125,54.96875,61.0,56.5,70.0625,66.4375,70.625,66.875,63.59375,71.0,64.375,61.9375,56.25,66.7323
7017,2518535,3709,1.000000,69.25,74.25,2.826172,1.212891,3.148438,65.762044,59.722245,66.481357,69.112275,58.764139,66.000838,60.53125,54.96875,61.0,56.5,70.0625,66.4375,70.625,66.875,63.59375,71.0,64.375,61.9375,56.25,66.7323


In [112]:
train_MF.to_csv('data/training_data/train_MF.csv')

# Création du train dataset

In [113]:
train_GK.head(1)

,game_id,player_club_id,GK,GK_overall,GK_potential,GK_skill_moves,GK_international_reputation,GK_weak_foot,GK_attacking_crossing,GK_attacking_heading_accuracy,GK_skill_curve,GK_skill_fk_accuracy,GK_movement_agility,GK_movement_reactions,GK_movement_balance,GK_power_shot_power,GK_power_jumping,GK_power_stamina,GK_mentality_aggression,GK_mentality_positioning,GK_mentality_penalties,GK_goalkeeping_diving,GK_goalkeeping_speed
0,2457642,610,1.0,77.0,77.0,1.0,2.0,3.0,25.0,25.0,25.0,25.0,68.0,79.0,58.0,23.0,82.0,42.0,34.0,25.0,29.0,82.0,58.0


In [114]:
train = pd.merge(train_GK , train_ATK, \
         on = ['player_club_id','game_id'], how = 'inner')


In [115]:
train = pd.merge(train , train_DEF, \
         on = ['player_club_id','game_id'], how = 'inner')
train = pd.merge(train , train_MF, \
         on = ['player_club_id','game_id'], how = 'inner')
train.head(1)

,game_id,player_club_id,GK,GK_overall,GK_potential,GK_skill_moves,GK_international_reputation,GK_weak_foot,GK_attacking_crossing,GK_attacking_heading_accuracy,GK_skill_curve,GK_skill_fk_accuracy,GK_movement_agility,GK_movement_reactions,GK_movement_balance,GK_power_shot_power,GK_power_jumping,GK_power_stamina,GK_mentality_aggression,GK_mentality_positioning,GK_mentality_penalties,GK_goalkeeping_diving,GK_goalkeeping_speed,attack,ATK_overall,ATK_potential,ATK_skill_moves,ATK_international_reputation,ATK_weak_foot,ATK_pace,ATK_shooting,ATK_passing,ATK_dribbling,ATK_defending,ATK_physic,ATK_attacking_crossing,ATK_attacking_heading_accuracy,ATK_skill_curve,ATK_skill_fk_accuracy,ATK_movement_agility,ATK_movement_reactions,ATK_movement_balance,ATK_power_shot_power,ATK_power_jumping,ATK_power_stamina,ATK_mentality_aggression,ATK_mentality_positioning,ATK_mentality_penalties,ATK_mentality_composure,defense,DEF_overall,DEF_potential,DEF_skill_moves,DEF_international_reputation,DEF_weak_foot,DEF_pace,DEF_shooting,DEF_passing,DEF_dribbling,DEF_defending,DEF_physic,DEF_attacking_crossing,DEF_attacking_heading_accuracy,DEF_skill_curve,DEF_skill_fk_accuracy,DEF_movement_agility,DEF_movement_reactions,DEF_movement_balance,DEF_power_shot_power,DEF_power_jumping,DEF_power_stamina,DEF_mentality_aggression,DEF_mentality_positioning,DEF_mentality_penalties,DEF_mentality_composure,midfield,MF_overall,MF_potential,MF_skill_moves,MF_international_reputation,MF_weak_foot,MF_pace,MF_shooting,MF_passing,MF_dribbling,MF_defending,MF_physic,MF_attacking_crossing,MF_attacking_heading_accuracy,MF_skill_curve,MF_skill_fk_accuracy,MF_movement_agility,MF_movement_reactions,MF_movement_balance,MF_power_shot_power,MF_power_jumping,MF_power_stamina,MF_mentality_aggression,MF_mentality_positioning,MF_mentality_penalties,MF_mentality_composure
0,2457642,610,1.0,77.0,77.0,1.0,2.0,3.0,25.0,25.0,25.0,25.0,68.0,79.0,58.0,23.0,82.0,42.0,34.0,25.0,29.0,82.0,58.0,3.000244,71.0,82.4375,3.0,1.0,3.21875,75.825439,62.260417,65.342611,72.450928,32.920817,58.145671,63.40625,62.34375,67.5625,55.28125,75.3125,66.25,66.625,68.8125,59.25,63.71875,45.53125,63.15625,55.96875,0.0,4.43335,72.125,76.0,2.224609,1.225586,3.126953,72.635077,43.77957,61.192126,67.447412,71.191575,70.417181,55.5,72.5625,48.5,31.375,66.5,70.1875,64.9375,61.84375,78.3125,73.0,70.0625,55.3125,40.0,0.0,2.566895,75.1875,78.8125,2.611328,1.389648,3.609375,73.495434,71.375761,71.571918,76.118721,54.801941,65.583143,62.34375,60.96875,68.9375,59.8125,75.3125,75.25,73.5,73.5625,63.375,78.8125,59.28125,74.6875,67.125,0.0


In [116]:
train.to_csv('data/training_data/train.csv')

# Générer le train.csv

In [127]:
train = pd.read_csv('data/training_data/train.csv', index_col=0)

In [116]:
# il faut réussir à diviser le dataset en home et away, puis faire un join.

In [117]:
#repérons les club_id qui n'existent pas
club_IDs = clubs.club_id.unique()
train.loc[~train.player_club_id.isin(club_IDs),'player_club_id'].unique()

array([-25110, -25814, -24335, -24305, -24262,  -4587])

In [75]:
tt = train.loc[train.player_club_id == -4587,'game_id'].values[2]
games.loc[games.game_id==tt,:]

,game_id,competition_code,season,round,date,home_club_id,away_club_id,home_club_goals,away_club_goals,home_club_position,away_club_position,stadium,attendance,referee,url
41624,3661873,GR1,2021,5. Matchday,2021-10-02,60949,6676,2,1,2.0,12.0,Panthessaliko Stadio,1881.0,Anastasios Papapetrou,https://www.transfermarkt.co.uk/volos-nps_aste...


1. -25110 => 40426
2. -25814 => 39722
3. -24335 => 41201
4. -24305 => 41231
5. -24262 => 41274
6. -4587 => 60949

In [129]:
#on a repéré quelle mauvaise notation correspondait à qui, maintenant changeons les valeurs:
train.loc[train.player_club_id == -25110,'player_club_id'] = 40426
train.loc[train.player_club_id == -25814,'player_club_id'] = 39722
train.loc[train.player_club_id == -24335,'player_club_id'] = 41201
train.loc[train.player_club_id == -24305,'player_club_id'] = 41231
train.loc[train.player_club_id == -24262,'player_club_id'] = 41274
train.loc[train.player_club_id == -4587,'player_club_id'] = 60949

In [130]:
#maintenant c'est bon!
train.loc[train.player_club_id <= 0,:]

,game_id,player_club_id,GK,GK_overall,GK_potential,GK_skill_moves,GK_international_reputation,GK_weak_foot,GK_attacking_crossing,GK_attacking_heading_accuracy,GK_skill_curve,GK_skill_fk_accuracy,GK_movement_agility,GK_movement_reactions,GK_movement_balance,GK_power_shot_power,GK_power_jumping,GK_power_stamina,GK_mentality_aggression,GK_mentality_positioning,GK_mentality_penalties,GK_goalkeeping_diving,GK_goalkeeping_speed,attack,ATK_overall,ATK_potential,ATK_skill_moves,ATK_international_reputation,ATK_weak_foot,ATK_pace,ATK_shooting,ATK_passing,ATK_dribbling,ATK_defending,ATK_physic,ATK_attacking_crossing,ATK_attacking_heading_accuracy,ATK_skill_curve,ATK_skill_fk_accuracy,ATK_movement_agility,ATK_movement_reactions,ATK_movement_balance,ATK_power_shot_power,ATK_power_jumping,ATK_power_stamina,ATK_mentality_aggression,ATK_mentality_positioning,ATK_mentality_penalties,ATK_mentality_composure,defense,DEF_overall,DEF_potential,DEF_skill_moves,DEF_international_reputation,DEF_weak_foot,DEF_pace,DEF_shooting,DEF_passing,DEF_dribbling,DEF_defending,DEF_physic,DEF_attacking_crossing,DEF_attacking_heading_accuracy,DEF_skill_curve,DEF_skill_fk_accuracy,DEF_movement_agility,DEF_movement_reactions,DEF_movement_balance,DEF_power_shot_power,DEF_power_jumping,DEF_power_stamina,DEF_mentality_aggression,DEF_mentality_positioning,DEF_mentality_penalties,DEF_mentality_composure,midfield,MF_overall,MF_potential,MF_skill_moves,MF_international_reputation,MF_weak_foot,MF_pace,MF_shooting,MF_passing,MF_dribbling,MF_defending,MF_physic,MF_attacking_crossing,MF_attacking_heading_accuracy,MF_skill_curve,MF_skill_fk_accuracy,MF_movement_agility,MF_movement_reactions,MF_movement_balance,MF_power_shot_power,MF_power_jumping,MF_power_stamina,MF_mentality_aggression,MF_mentality_positioning,MF_mentality_penalties,MF_mentality_composure


In [131]:
def HomeOrAway(x):
    if games.loc[games.game_id == x['game_id'],'home_club_id'].values[0] == x['player_club_id']:
        return 'Home'
    elif games.loc[games.game_id == x['game_id'],'away_club_id'].values[0] == x['player_club_id']:
        return 'Away'
    else:
        return 'problem'    

In [132]:
train['HomeOrAway']=None
train['HomeOrAway'] = train.progress_apply(HomeOrAway, axis = 1)


100%|██████████| 59090/59090 [00:27<00:00, 2164.91it/s]


In [133]:
train_home = train.loc[train.HomeOrAway == 'Home',:].drop(columns = ['HomeOrAway','player_club_id']).copy()
train_away = train.loc[train.HomeOrAway == 'Away',:].drop(columns = ['HomeOrAway','player_club_id']).copy()

train_home.rename(columns={x:'H_'+x for x in train_home.columns[1:]}, inplace = True)
train_away.rename(columns={x:'A_'+x for x in train_away.columns[1:]}, inplace = True)

In [134]:
train_final = pd.merge(train_home , train_away, on = ['game_id'], how = 'inner')
train_final

,game_id,H_GK,H_GK_overall,H_GK_potential,H_GK_skill_moves,H_GK_international_reputation,H_GK_weak_foot,H_GK_attacking_crossing,H_GK_attacking_heading_accuracy,H_GK_skill_curve,H_GK_skill_fk_accuracy,H_GK_movement_agility,H_GK_movement_reactions,H_GK_movement_balance,H_GK_power_shot_power,H_GK_power_jumping,H_GK_power_stamina,H_GK_mentality_aggression,H_GK_mentality_positioning,H_GK_mentality_penalties,H_GK_goalkeeping_diving,H_GK_goalkeeping_speed,H_attack,H_ATK_overall,H_ATK_potential,H_ATK_skill_moves,H_ATK_international_reputation,H_ATK_weak_foot,H_ATK_pace,H_ATK_shooting,H_ATK_passing,H_ATK_dribbling,H_ATK_defending,H_ATK_physic,H_ATK_attacking_crossing,H_ATK_attacking_heading_accuracy,H_ATK_skill_curve,H_ATK_skill_fk_accuracy,H_ATK_movement_agility,H_ATK_movement_reactions,H_ATK_movement_balance,H_ATK_power_shot_power,H_ATK_power_jumping,H_ATK_power_stamina,H_ATK_mentality_aggression,H_ATK_mentality_positioning,H_ATK_mentality_penalties,H_ATK_mentality_composure,H_defense,H_DEF_overall,H_DEF_potential,H_DEF_skill_moves,H_DEF_international_reputation,H_DEF_weak_foot,H_DEF_pace,H_DEF_shooting,H_DEF_passing,H_DEF_dribbling,H_DEF_defending,H_DEF_physic,H_DEF_attacking_crossing,H_DEF_attacking_heading_accuracy,H_DEF_skill_curve,H_DEF_skill_fk_accuracy,H_DEF_movement_agility,H_DEF_movement_reactions,H_DEF_movement_balance,H_DEF_power_shot_power,H_DEF_power_jumping,H_DEF_power_stamina,H_DEF_mentality_aggression,H_DEF_mentality_positioning,H_DEF_mentality_penalties,H_DEF_mentality_composure,H_midfield,H_MF_overall,H_MF_potential,H_MF_skill_moves,H_MF_international_reputation,H_MF_weak_foot,H_MF_pace,H_MF_shooting,H_MF_passing,H_MF_dribbling,H_MF_defending,H_MF_physic,H_MF_attacking_crossing,H_MF_attacking_heading_accuracy,H_MF_skill_curve,H_MF_skill_fk_accuracy,H_MF_movement_agility,H_MF_movement_reactions,H_MF_movement_balance,H_MF_power_shot_power,H_MF_power_jumping,H_MF_power_stamina,H_MF_mentality_aggression,H_MF_mentality_positioning,H_MF_mentality_penalties,H_MF_mentality_composure,A_GK,A_GK_overall,A_GK_potential,A_GK_skill_moves,A_GK_international_reputation,A_GK_weak_foot,A_GK_attacking_crossing,A_GK_attacking_heading_accuracy,A_GK_skill_curve,A_GK_skill_fk_accuracy,A_GK_movement_agility,A_GK_movement_reactions,A_GK_movement_balance,A_GK_power_shot_power,A_GK_power_jumping,A_GK_power_stamina,A_GK_mentality_aggression,A_GK_mentality_positioning,A_GK_mentality_penalties,A_GK_goalkeeping_diving,A_GK_goalkeeping_speed,A_attack,A_ATK_overall,A_ATK_potential,A_ATK_skill_moves,A_ATK_international_reputation,A_ATK_weak_foot,A_ATK_pace,A_ATK_shooting,A_ATK_passing,A_ATK_dribbling,A_ATK_defending,A_ATK_physic,A_ATK_attacking_crossing,A_ATK_attacking_heading_accuracy,A_ATK_skill_curve,A_ATK_skill_fk_accuracy,A_ATK_movement_agility,A_ATK_movement_reactions,A_ATK_movement_balance,A_ATK_power_shot_power,A_ATK_power_jumping,A_ATK_power_stamina,A_ATK_mentality_aggression,A_ATK_mentality_positioning,A_ATK_mentality_penalties,A_ATK_mentality_composure,A_defense,A_DEF_overall,A_DEF_potential,A_DEF_skill_moves,A_DEF_international_reputation,A_DEF_weak_foot,A_DEF_pace,A_DEF_shooting,A_DEF_passing,A_DEF_dribbling,A_DEF_defending,A_DEF_physic,A_DEF_attacking_crossing,A_DEF_attacking_heading_accuracy,A_DEF_skill_curve,A_DEF_skill_fk_accuracy,A_DEF_movement_agility,A_DEF_movement_reactions,A_DEF_movement_balance,A_DEF_power_shot_power,A_DEF_power_jumping,A_DEF_power_stamina,A_DEF_mentality_aggression,A_DEF_mentality_positioning,A_DEF_mentality_penalties,A_DEF_mentality_composure,A_midfield,A_MF_overall,A_MF_potential,A_MF_skill_moves,A_MF_international_reputation,A_MF_weak_foot,A_MF_pace,A_MF_shooting,A_MF_passing,A_MF_dribbling,A_MF_defending,A_MF_physic,A_MF_attacking_crossing,A_MF_attacking_heading_accuracy,A_MF_skill_curve,A_MF_skill_fk_accuracy,A_MF_movement_agility,A_MF_movement_reactions,A_MF_movement_balance,A_MF_power_shot_power,A_MF_power_jumping,A_MF_power_stamina,A_MF_mentality_aggression,A_MF_mentality_positioning,A_MF_mentality_pe

In [124]:
x = train_final.game_id[4]
games.loc[games.game_id==x,'home_club_goals'].values[0]-games.loc[games.game_id==x,'away_club_goals'].values[0]

-1

In [135]:
#On rajoute les résultats sous la forme de la différence de buts (du point de vue de l'équipe Home)
train_final['Home_result'] = train_final.game_id.progress_apply(lambda x:
                                                                games.loc[games.game_id==x,'home_club_goals'].values[0]-games.loc[games.game_id==x,'away_club_goals'].values[0]
                                                               )

100%|██████████| 29545/29545 [00:16<00:00, 1836.14it/s]


In [136]:
train_final = train_final.astype(dtype={x: np.float16 for x in train_final.columns[1:-1]})
train_final.to_csv('data/training_data/train_final.csv')